In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
print(df)

             artist                   song  \
0              ABBA  Ahe's My Kind Of Girl   
1              ABBA       Andante, Andante   
2              ABBA         As Good As New   
3              ABBA                   Bang   
4              ABBA       Bang-A-Boomerang   
...             ...                    ...   
57645  Ziggy Marley          Good Old Days   
57646  Ziggy Marley          Hand To Mouth   
57647          Zwan           Come With Me   
57648          Zwan                 Desire   
57649          Zwan              Heartsong   

                                              link  \
0       /a/abba/ahes+my+kind+of+girl_20598417.html   
1            /a/abba/andante+andante_20002708.html   
2             /a/abba/as+good+as+new_20003033.html   
3                       /a/abba/bang_20598415.html   
4           /a/abba/bang+a+boomerang_20002668.html   
...                                            ...   
57645  /z/ziggy+marley/good+old+days_10198588.html   
57646  /z/ziggy

In [4]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [5]:
df=df.sample(5000).drop('link',axis=1).reset_index(drop=True)

In [7]:
df['text'][0]

"It's in every one of us to be wise  \r\nFind your heart  \r\nAnd open up both your eyes  \r\nWe can all know every thing  \r\nWithout ever knowing why  \r\nIt's in every one of us by and by  \r\nIt's in every one of us  \r\nFind your heart  \r\nAnd open up both your eyes  \r\nWe can all know every thing  \r\nWithout ever knowing why  \r\nIt's in every one of us by and by  \r\nIt's in every one of us  \r\nI just remembered  \r\nIt's like I've been sleeping for years  \r\nI'm not awake as I can be  \r\nBut my seeing's better  \r\nI can see through the tears  \r\nI've been realizing that  \r\nI bought this ticket  \r\nAnd watching only half of the show  \r\nBut there's scenery and lights  \r\nAnd a cast of thousands  \r\nWho all know what I know  \r\nAnd it's good that it's so  \r\nIt's in every one of us to be wise  \r\nFind your heart  \r\nAnd open up both your eyes  \r\nWe can all know every thing  \r\nWithout ever knowing why  \r\nIt's in every one of us by and by  \r\nIt's in every 

In [8]:
df['text']=df['text'].str.lower().replace(r'^\w\s','').replace(r'\n','',regex=True)

In [9]:
import nltk

In [11]:
from nltk.stem import PorterStemmer
nltk.download('punkt')
stemmer = PorterStemmer()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\94783\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
def token(txt):
    token=nltk.word_tokenize(txt)
    a=[stemmer.stem(w) for w in token]
    return " ".join(a)

In [20]:
df['text'].apply(lambda x:token(x))

0       it 's in everi one of us to be wise find your ...
1       believ in the supernatur believ in the superna...
2       everybodi know that i 'm just a joe that like ...
3       they pave paradis and put up a park lot with a...
4       the other day i came across someon i knew and ...
                              ...                        
4995    half moon , night time sky , seven star , heav...
4996    i climb the mountain top , i saw the bottom dr...
4997    someth 's goin ' on i do n't know what it is b...
4998    they say you came from heaven but i know that ...
4999    i use to bite my tongu and hold my breath scar...
Name: text, Length: 5000, dtype: object

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
tfid=TfidfVectorizer(analyzer='word',stop_words='english')

In [23]:
matrix=tfid.fit_transform(df['text'])

In [36]:
similar=cosine_similarity(matrix)
similar[0]

array([1.        , 0.00553816, 0.03188874, ..., 0.04137105, 0.05366043,
       0.00204358])

In [42]:
df[df['song'] == 'Nobody But You']


,artist,song,text
2,James Taylor,Nobody But You,everybody knows that i'm just a joe that likes...


In [38]:
df


,artist,song,text
0,Cliff Richard,It's In Every One Of Us,it's in every one of us to be wise \rfind you...
1,Air Supply,Believe In The Supernatural,believe in the supernatural \rbelieve in the ...
2,James Taylor,Nobody But You,everybody knows that i'm just a joe that likes...
3,Amy Grant,Big Yellow Taxi,they paved paradise and put up a parking lot ...
4,New Order,Weirdo,the other day i came across \rsomeone i knew ...
...,...,...,...
4995,Janis Joplin,Half Moon,"half moon, night time sky, \rseven stars, hea..."
4996,Lady Gaga,Ink,"i climb the mountain top, i saw the bottom dro..."
4997,Yes,Abilene,something's goin' on \ri don't know what it i...
4998,Black Sabbath,Rusty Angels,they say you came from heaven but i know that'...


In [61]:
from sklearn.metrics.pairwise import cosine_similarity

def recommender(Song_name):
    idx = df[df['song'] == Song_name].index[0]
    distances = sorted(list(enumerate(similar[idx])), reverse=True, key=lambda x: x[1])
    songs = []
    for s_id in range(1, 6):
        songs.append(df.iloc[distances[s_id][0]].song)
    return songs


   

In [62]:
recommender("Half Moon")

['When We Stand Together',
 'Yeah Yeah',
 'Fantasy Man',
 'Slicker Drips',
 'Louie Louie']

In [71]:
import pickle
pickle.dump(similar,open("Similarity","wb"))
pickle.dump(df,open("df","wb"))


